In [1]:
import requests
import pandas as pd
import datetime

In [2]:
import xmltodict
import json

In [3]:
import re

## Daten herunterladen von API und umwandeln

In [4]:
#Abfrage für alle Vorstösse einer Partei im gewählten Zeitrahmen (1.1.14 bis 1.1.24)
r = requests.get('http://www.gemeinderat-zuerich.ch/api/geschaeft/searchdetails?q=partei all "SVP" AND beginn_start > "2014-01-01 00:00:00" AND beginn_start < "2024-01-01 00:00:00" sortBy beginn_start/sort.ascending&l=de-CH')

In [5]:
#Die Arbeit mit den XML-Daten funktioniert schlecht, darum umwandeln in JSON
xml_data = r.text
data_dict = xmltodict.parse(xml_data)

## DataFrame mit Vorstössen erstellen

In [6]:
# Minidict mit den wichtigsten Angaben zu den Vorstössen. Da es nicht immer einen Mitunterzeichner gibt, braucht's einen Fail-Safe
vorstoesse = []

for hit in data_dict["SearchDetailResponse"]["Hit"]:
    titel = hit["Geschaeft"]["Titel"]
    geschaeftsart = hit["Geschaeft"]["Geschaeftsart"]
    ID = hit["Geschaeft"]["@OBJ_GUID"]
    datum = hit["Geschaeft"]["Beginn"]["Start"]["#text"]
    erstunterzeichner = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Name"]
    partei_erst = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Partei"]
    
    # Kontrollieren ob "Mitunterzeichner" existiert
    mitunterzeichner_data = hit["Geschaeft"].get("Mitunterzeichner")
    
    if mitunterzeichner_data:
        # Wenn es einen Mitunterzeichner gibt, dann Daten herunterladen
        mitunterzeichner = mitunterzeichner_data["KontaktGremium"]["Name"]
        partei_mit = mitunterzeichner_data["KontaktGremium"]["Partei"]
    else:
        # Wenn nicht, dann None einsetzen
        mitunterzeichner = None
        partei_mit = None
    
    status = hit["Geschaeft"]["Geschaeftsstatus"]
    
    mini_dict = {"Titel": titel,
                 "Geschäftsart": geschaeftsart,
                 "ID": ID,
                 "Datum": datum,
                 "Erstunterzeichner": erstunterzeichner,
                 "Partei Erstunterzeichner": partei_erst,
                 "Mitunterzeichner": mitunterzeichner,
                 "Partei Mitunterzeichner": partei_mit,
                 "Geschäftsstatus": status}
    
    vorstoesse.append(mini_dict)

In [7]:
df_partei = pd.DataFrame(vorstoesse)

In [8]:
df_partei

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus
0,Beiträge und Dienstleistungen für mittellose H...,Schriftliche Anfrage,9300f1f882cd4ece9b634ac694c1a1e1,2014-01-08T00:00:00.000,Roland Scheck,SVP,Urs Fehr,SVP,Abgeschlossen
1,Ausstellung «Dada x Statistik» im Cabaret Volt...,Schriftliche Anfrage,0ec0dfeb654d4fed99acc19aef7b6373,2014-01-22T00:00:00.000,Daniel Regli,SVP,Thomas Schwendener,SVP,Abgeschlossen
2,"Asylunterkunft am Wydäckerring, Hintergründe z...",Schriftliche Anfrage,921e1b3c08894f8cb108f8a7245dd787,2014-01-29T00:00:00.000,Margrit Haller,SVP,Roger Bartholdi,SVP,Abgeschlossen
3,"Brand in der Roten Fabrik im Jahr 2012, finanz...",Interpellation,56d2c264de5542ad9749e10dd0c3ceb0,2014-02-05T00:00:00.000,Daniel Regli,SVP,Thomas Schwendener,SVP,Abgeschlossen
4,Exklusive Vergabe der Ticketrechte der Hallens...,Schriftliche Anfrage,7ca5289be14b47a89af6e425faafbb91,2014-02-26T00:00:00.000,Roland Scheck,SVP,Roger Liebi,SVP,Abgeschlossen
...,...,...,...,...,...,...,...,...,...
649,Verzicht auf die Erhöhung der Taxen in den stä...,Schriftliche Anfrage,9b74321c1fd047c9844224f8e487aca5,2023-11-29T00:00:00.000,Walter Anken,SVP,Samuel Balsiger,SVP,InBearbeitung
650,"Sportanlage Oerlikon, Bericht zu den «Lessons ...",Postulat,fdbb5ebb7fed4bb394ce0342a6293029,2023-11-29T00:00:00.000,Martin Götzl,SVP,Reto Brüesch,SVP,InBearbeitung
651,Verzicht auf den Einsatz von elektronischen Au...,Postulat,e541ea96e7c340c99d6d0c2c5e7c3ef6,2023-12-06T00:00:00.000,Derek Richter,SVP,Stephan Iten,SVP,InBearbeitung
652,"Ertrag aus Ordnungsbussen, Reduzierung des bud...",Postulat,b52db7bc3ce14d8c89215fd22e65478f,2023-12-06T00:00:00.000,Bernhard im Oberdorf,SVP,Martin Götzl,SVP,Abgeschlossen


## Resultate für Motionen und Postulate herunterladen

In [9]:
#Hiermit hole ich mir die Resultate für die Motionen und Postulate. 
#Es braucht einen doppelten Loop und Problembehebungsmassnahmen

entscheide = []

for hit in data_dict['SearchDetailResponse']['Hit']:
    # Kontrollieren, ob Ablaufschritte existiert und eine Liste ist
    ablaufschritte = hit['Geschaeft'].get('Ablaufschritte', {}).get('Aufgabe', [])
    
    if not isinstance(ablaufschritte, list):
        # Falls Aufgabe keine Liste ist, wird sie hier in eine Liste umgewandelt
        ablaufschritte = [ablaufschritte]
    
    for aufgabe in ablaufschritte:
        # Kontrollieren, ob AblaufschrittName existiert
        entscheid = aufgabe.get('AblaufschrittName')
        if entscheid is not None:
            entscheide.append({
                "HitID": hit["Geschaeft"]["@OBJ_GUID"],
                "Entscheid": entscheid,
            })

df_entscheid = pd.DataFrame(entscheide)

In [10]:
df_entscheid

,HitID,Entscheid
0,9300f1f882cd4ece9b634ac694c1a1e1,"Eingang, Frist 3 Monate"
1,9300f1f882cd4ece9b634ac694c1a1e1,"Stadtrat, Antwort"
2,9300f1f882cd4ece9b634ac694c1a1e1,Kenntnisnahme
3,0ec0dfeb654d4fed99acc19aef7b6373,"Eingang, Frist 3 Monate"
4,0ec0dfeb654d4fed99acc19aef7b6373,"Stadtrat, Antwort"
...,...,...
2177,b52db7bc3ce14d8c89215fd22e65478f,"Stadtrat, Ablehnung"
2178,b52db7bc3ce14d8c89215fd22e65478f,Ablehnung
2179,287136dec2c44a8ebb8ae31eddd8427d,"Eingang, Frist 3 Monate"
2180,287136dec2c44a8ebb8ae31eddd8427d,"Stadtrat, Entgegennahme"


## Daten putzen

In [11]:
#Zu jedem Geschäft gibt es mehrere Verfahrensschritte. 
#Mich interessieren nur die Schlussentscheide, die ich mit den keywords heraushole

keywords = ["Überweisung", "Ablehnung", "Rückzug", "Umwandlung"]

# Ich kann ein Pattern kreieren mit meinen keyword, zusammengefügt werden sie mit dem OR operator (|)
pattern = '|'.join(map(re.escape, keywords))

# Jetzt kann ich das Pattern auf meine Entscheidspalte anwenden mit str.contains
df_entscheid_neu = df_entscheid[df_entscheid.Entscheid.str.contains(pattern, case=False, regex=True)]

In [12]:
df_entscheid_neu

,HitID,Entscheid
26,2c561e715fd64a409a47bf58ef09214d,"Überweisung, Frist 24 Monate"
34,ecad9c23e08b4fa297d5399b6b43bf49,"Überweisung, Frist 24 Monate"
61,714ec5ab53464ffdb8b4e7f0a24908af,"Überweisung, Frist 24 Monate"
67,7506e23a71fd4cbebbcc1a8d1f4c57e0,Ablehnung
69,7506e23a71fd4cbebbcc1a8d1f4c57e0,"Stadtrat, Ablehnung"
...,...,...
2174,e541ea96e7c340c99d6d0c2c5e7c3ef6,"Stadtrat, Ablehnung"
2175,e541ea96e7c340c99d6d0c2c5e7c3ef6,Ablehnung
2177,b52db7bc3ce14d8c89215fd22e65478f,"Stadtrat, Ablehnung"
2178,b52db7bc3ce14d8c89215fd22e65478f,Ablehnung


In [13]:
# Das Problem ist noch nicht ganz gelöst, zwei Verfahrensschritte sind geblieben, die ich nicht brauche: 
exclude_values = ["Stadtrat, Ablehnung", "Ablehnung, beantragt"]

# Für das bereinigte DataFrame behalte ich nun alle Zeilen, welche die exclude-values nicht enthalten
df_entscheid_clean = df_entscheid_neu[~df_entscheid_neu['Entscheid'].isin(exclude_values)]

In [14]:
df_entscheid_clean

,HitID,Entscheid
26,2c561e715fd64a409a47bf58ef09214d,"Überweisung, Frist 24 Monate"
34,ecad9c23e08b4fa297d5399b6b43bf49,"Überweisung, Frist 24 Monate"
61,714ec5ab53464ffdb8b4e7f0a24908af,"Überweisung, Frist 24 Monate"
67,7506e23a71fd4cbebbcc1a8d1f4c57e0,Ablehnung
80,0d0ec40406b04adcbe385820e5931be4,Ablehnung
...,...,...
2164,51668c3567644bf3acaa68d36db4f56a,"Überweisung, Frist 24 Monate"
2168,96fbcf064e3646d3b40d29ddc48475c3,Ablehnung
2175,e541ea96e7c340c99d6d0c2c5e7c3ef6,Ablehnung
2178,b52db7bc3ce14d8c89215fd22e65478f,Ablehnung


In [15]:
#Nun sind noch verschiedene Varianten für Überweisung verblieben, die ich noch vereinheitliche und in neuer Spalte speichere
def clean(value):
    if 'Überweisung' in value:
        return 'überweisung'
    
    return value

#df_entscheid_clean["_entscheidbereinigt"] = df_entscheid_clean["Entscheid"].apply(clean)
df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']
df_entscheid_clean.loc[df_entscheid_clean['_entscheidbereinigt'].str.contains("Überweisung"), '_entscheidbereinigt'] = "Überweisung"
df_entscheid_clean

C:\Users\hudecj\AppData\Local\Temp\ipykernel_89296\3655237080.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']


,HitID,Entscheid,_entscheidbereinigt
26,2c561e715fd64a409a47bf58ef09214d,"Überweisung, Frist 24 Monate",Überweisung
34,ecad9c23e08b4fa297d5399b6b43bf49,"Überweisung, Frist 24 Monate",Überweisung
61,714ec5ab53464ffdb8b4e7f0a24908af,"Überweisung, Frist 24 Monate",Überweisung
67,7506e23a71fd4cbebbcc1a8d1f4c57e0,Ablehnung,Ablehnung
80,0d0ec40406b04adcbe385820e5931be4,Ablehnung,Ablehnung
...,...,...,...
2164,51668c3567644bf3acaa68d36db4f56a,"Überweisung, Frist 24 Monate",Überweisung
2168,96fbcf064e3646d3b40d29ddc48475c3,Ablehnung,Ablehnung
2175,e541ea96e7c340c99d6d0c2c5e7c3ef6,Ablehnung,Ablehnung
2178,b52db7bc3ce14d8c89215fd22e65478f,Ablehnung,Ablehnung


In [16]:
#Hier führe ich die beiden Tabellen zusammen anhand der ID resp. HitID

df_neu = df_partei.copy()

df_partei_final = df_neu.merge(df_entscheid_clean, how='left', left_on='ID', right_on='HitID')[["Titel", "Geschäftsart", "ID", "Datum", "Erstunterzeichner", "Partei Erstunterzeichner", "Mitunterzeichner", "Partei Mitunterzeichner", "Geschäftsstatus", "Entscheid", '_entscheidbereinigt']]
df_partei_final

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,Beiträge und Dienstleistungen für mittellose H...,Schriftliche Anfrage,9300f1f882cd4ece9b634ac694c1a1e1,2014-01-08T00:00:00.000,Roland Scheck,SVP,Urs Fehr,SVP,Abgeschlossen,NaN,NaN
1,Ausstellung «Dada x Statistik» im Cabaret Volt...,Schriftliche Anfrage,0ec0dfeb654d4fed99acc19aef7b6373,2014-01-22T00:00:00.000,Daniel Regli,SVP,Thomas Schwendener,SVP,Abgeschlossen,NaN,NaN
2,"Asylunterkunft am Wydäckerring, Hintergründe z...",Schriftliche Anfrage,921e1b3c08894f8cb108f8a7245dd787,2014-01-29T00:00:00.000,Margrit Haller,SVP,Roger Bartholdi,SVP,Abgeschlossen,NaN,NaN
3,"Brand in der Roten Fabrik im Jahr 2012, finanz...",Interpellation,56d2c264de5542ad9749e10dd0c3ceb0,2014-02-05T00:00:00.000,Daniel Regli,SVP,Thomas Schwendener,SVP,Abgeschlossen,NaN,NaN
4,Exklusive Vergabe der Ticketrechte der Hallens...,Schriftliche Anfrage,7ca5289be14b47a89af6e425faafbb91,2014-02-26T00:00:00.000,Roland Scheck,SVP,Roger Liebi,SVP,Abgeschlossen,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
649,Verzicht auf die Erhöhung der Taxen in den stä...,Schriftliche Anfrage,9b74321c1fd047c9844224f8e487aca5,2023-11-29T00:00:00.000,Walter Anken,SVP,Samuel Balsiger,SVP,InBearbeitung,NaN,NaN
650,"Sportanlage Oerlikon, Bericht zu den «Lessons ...",Postulat,fdbb5ebb7fed4bb394ce0342a6293029,2023-11-29T00:00:00.000,Martin Götzl,SVP,Reto Brüesch,SVP,InBearbeitung,NaN,NaN
651,Verzicht auf den Einsatz von elektronischen Au...,Postulat,e541ea96e7c340c99d6d0c2c5e7c3ef6,2023-12-06T00:00:00.000,Derek Richter,SVP,Stephan Iten,SVP,InBearbeitung,Ablehnung,Ablehnung
652,"Ertrag aus Ordnungsbussen, Reduzierung des bud...",Postulat,b52db7bc3ce14d8c89215fd22e65478f,2023-12-06T00:00:00.000,Bernhard im Oberdorf,SVP,Martin Götzl,SVP,Abgeschlossen,Ablehnung,Ablehnung


In [17]:
#Diese Resultate speichern
df_partei_final.to_csv('SVP_Vorstoesse_14_23_mit_Resultat.csv', index=False)

## Das Problem mit den umgewandelten Postulaten lösen

In [18]:
#Ein Problem bleibt noch: Bei Motionen, die in Postulate umgewandelt wurden, ist das Ergebnis nicht verzeichnet. 
#Weil es wenige Fälle sind, schaue ich das von Hand in den dazugehörigen PDF nach.
df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
159,"Gewerbefahrzeuge, Befreiung von den Parkgebühr...",Motion,c5911c914eab4f6aa3309a699e523761,2017-05-10T00:00:00.000,Stephan Iten,SVP,Stefan Urech,SVP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
326,Umnutzung der Parzelle SE 6364 (Zihlacker) zur...,Motion,3d8ff8c6f78f4ef98348ccf21770e439,2020-02-05T00:00:00.000,Martin Götzl,SVP,Thomas Schwendener,SVP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
424,Zonenplanänderung für den Ersatz der wegfallen...,Motion,b06d0b432d8041c599e15f204da4c5ba,2021-05-26T00:00:00.000,Reto Brüesch,SVP,Sabine Koch,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
452,Sicherstellung eines Anteils von mindestens 20...,Motion,8c6c4fe8f4b34d008ae3d1ac593e045d,2021-12-01T00:00:00.000,Reto Brüesch,SVP,Ernst Danner,EVP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat


In [19]:
#Um schneller zum Ziel zu kommen, lasse ich mir die urls zu den Geschäften automatisch erstellen
df_umwandlung = df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

In [20]:
url_liste = []
base = "https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid="

for elem in df_umwandlung["ID"]:
    url_liste.append(base + elem)
    

In [21]:
url_liste

['https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=c5911c914eab4f6aa3309a699e523761',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=3d8ff8c6f78f4ef98348ccf21770e439',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=b06d0b432d8041c599e15f204da4c5ba',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=8c6c4fe8f4b34d008ae3d1ac593e045d']

In [22]:
#In einem ersten Schritt passe ich nun die umgewandelten Postulate an, die abgelehnt wurden
#Mit dem loc-Befehl kann man über die Index-Nr. die richtige Zeile anwählen und dann die Spalte
df_test = df_partei_final.copy()
df_test.loc[326, "_entscheidbereinigt"] = "Ablehnung"
df_test.loc[452, "_entscheidbereinigt"] = "Ablehnung"


In [23]:
#Ein Kontrollblick
pd.set_option('display.max_rows', None)
df_test

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,Beiträge und Dienstleistungen für mittellose H...,Schriftliche Anfrage,9300f1f882cd4ece9b634ac694c1a1e1,2014-01-08T00:00:00.000,Roland Scheck,SVP,Urs Fehr,SVP,Abgeschlossen,NaN,NaN
1,Ausstellung «Dada x Statistik» im Cabaret Volt...,Schriftliche Anfrage,0ec0dfeb654d4fed99acc19aef7b6373,2014-01-22T00:00:00.000,Daniel Regli,SVP,Thomas Schwendener,SVP,Abgeschlossen,NaN,NaN
2,"Asylunterkunft am Wydäckerring, Hintergründe z...",Schriftliche Anfrage,921e1b3c08894f8cb108f8a7245dd787,2014-01-29T00:00:00.000,Margrit Haller,SVP,Roger Bartholdi,SVP,Abgeschlossen,NaN,NaN
3,"Brand in der Roten Fabrik im Jahr 2012, finanz...",Interpellation,56d2c264de5542ad9749e10dd0c3ceb0,2014-02-05T00:00:00.000,Daniel Regli,SVP,Thomas Schwendener,SVP,Abgeschlossen,NaN,NaN
4,Exklusive Vergabe der Ticketrechte der Hallens...,Schriftliche Anfrage,7ca5289be14b47a89af6e425faafbb91,2014-02-26T00:00:00.000,Roland Scheck,SVP,Roger Liebi,SVP,Abgeschlossen,NaN,NaN
5,"Kleinkinderbetreuungsbeiträge (KKBB), Richtlin...",Schriftliche Anfrage,d7fbf886aeef462fb30b36977cad37df,2014-03-05T00:00:00.000,Roland Scheck,SVP,Mauro Tuena,SVP,Abgeschlossen,NaN,NaN
6,Geschwindigkeitsbeschränkungen auf der Dreiwie...,Schriftliche Anfrage,936c58409cb44502b05d65726d618fe9,2014-03-19T00:00:00.000,Roland Scheck,SVP,Kurt Hüssy,SVP,Abgeschlossen,NaN,NaN
7,"Illegal besetzte Liegenschaften, Hintergründe ...",Schriftliche Anfrage,69200f3d510c44d89ff1a6a513d8c2da,2014-04-16T00:00:00.000,Urs Fehr,SVP,Mauro Tuena,SVP,Abgeschlossen,NaN,NaN
8,Parkplätze bei der Kirche Unterdorf in Zürich-...,Postulat,2c561e715fd64a409a47bf58ef09214d,2014-06-04T00:00:00.000,Daniel Regli,SVP,Roberto Bertozzi,SVP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
9,"Schulhaus Buhnrain, Hintergründe zu den Konfli...",Schriftliche Anfrage,869c9e661b4f4327aa5454f66a31c164,2014-06-04T00:00:00.000,Daniel Regli,SVP,Martin Götzl,SVP,Abgeschlossen,NaN,NaN


In [24]:
pd.reset_option('display.max_rows')

In [25]:
#Nun kann ich alle übrigen umgewandelten Postulate zu "Überweisung" ändern
df_test['_entscheidbereinigt'] = df_test['_entscheidbereinigt'].replace('Umwandlung in Postulat', 'Überweisung')

In [26]:
#Diese Resultate speichern
df_test.to_csv('SVP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv', index=False)